## Simple RNN

In [1]:
import torch
import torch.nn as nn
import torch.optim as optim

# Set random seed for reproducibility
torch.manual_seed(42)

# Define a simple RNN model
class SimpleRNN(nn.Module):
    def __init__(self, input_size, hidden_size, output_size):
        super(SimpleRNN, self).__init__()

        self.hidden_size = hidden_size

        # RNN layer
        self.rnn = nn.RNN(input_size, hidden_size, batch_first=True)

        # Output layer
        self.fc = nn.Linear(hidden_size, output_size)

    def forward(self, x, hidden=None):
        # Initialize hidden state if not provided
        if hidden is None:
            hidden = torch.zeros(1, x.size(0), self.hidden_size)

        # Forward pass through RNN
        output, hidden = self.rnn(x, hidden)

        # Apply output layer to the last time step
        output = self.fc(output[:, -1, :])

        return output, hidden

# Example parameters
input_size = 1    # Dimension of input feature
hidden_size = 10  # Size of hidden layer
output_size = 1   # Dimension of output
batch_size = 5    # Number of sequences per batch
seq_length = 20   # Length of each sequence

# Create random data
X = torch.randn(batch_size, seq_length, input_size)
y = torch.randn(batch_size, output_size)

# Initialize model
model = SimpleRNN(input_size, hidden_size, output_size)

# Define loss function and optimizer
criterion = nn.MSELoss()
optimizer = optim.Adam(model.parameters(), lr=0.01)

# Train the model for a few epochs
num_epochs = 100

for epoch in range(num_epochs):
    # Forward pass
    output, hidden = model(X)

    # Compute loss
    loss = criterion(output, y)

    # Backward pass and optimize
    optimizer.zero_grad()
    loss.backward()
    optimizer.step()

    # Print progress
    if (epoch + 1) % 20 == 0:
        print(f'Epoch [{epoch+1}/{num_epochs}], Loss: {loss.item():.4f}')

# Make a prediction
with torch.no_grad():
    test_input = torch.randn(1, seq_length, input_size)
    prediction, _ = model(test_input)
    print(f'Prediction: {prediction.item():.4f}')

Epoch [20/100], Loss: 0.2921
Epoch [40/100], Loss: 0.0685
Epoch [60/100], Loss: 0.0047
Epoch [80/100], Loss: 0.0004
Epoch [100/100], Loss: 0.0001
Prediction: 0.5044


## Vocabulary Example

In [5]:
texts = [
    "I took the dog for a walk this morning",
    "The quick brown fox jumps over the lazy dog",
    "Hello how are you doing today"
]

vocab = set()
for text in texts:
    for word in text.lower().split():
        vocab.add(word)
vocab = list(vocab)
word_to_idx = {word: i for i, word in enumerate(vocab)}

print(f"Vocabulary: {vocab}")
print(f"Word to index mapping: {word_to_idx}")


Vocabulary: ['walk', 'lazy', 'for', 'this', 'fox', 'are', 'a', 'took', 'morning', 'quick', 'you', 'over', 'the', 'brown', 'hello', 'how', 'i', 'doing', 'dog', 'jumps', 'today']
Word to index mapping: {'walk': 0, 'lazy': 1, 'for': 2, 'this': 3, 'fox': 4, 'are': 5, 'a': 6, 'took': 7, 'morning': 8, 'quick': 9, 'you': 10, 'over': 11, 'the': 12, 'brown': 13, 'hello': 14, 'how': 15, 'i': 16, 'doing': 17, 'dog': 18, 'jumps': 19, 'today': 20}


## Tokenization Example


In [7]:

# Simple word tokenization
tokens = "I love this movie!".lower().split()
# Result: ['i', 'love', 'this', 'movie!']

# Better word tokenization (removing punctuation)
import re
tokens = re.findall(r'\w+', "I love this movie!".lower())
# Result: ['i', 'love', 'this', 'movie']

## Word-to-Index Conversion Example

In [2]:
def text_to_indices(text, word_to_idx, unknown_token_idx=0):
  """Converts a text string to a list of numerical indices.

  Args:
    text: The input text string.
    word_to_idx: A dictionary mapping words to their indices.
    unknown_token_idx: The index to use for words not found in the vocabulary.

  Returns:
    A list of indices representing the input text.
  """
  return [word_to_idx.get(word.lower(), unknown_token_idx) for word in text.split()]

# Example usage:
texts = [
    "I took the dog for a walk this morning",
    "The quick brown fox jumps over the lazy dog",
    "Hello how are you doing today"
]

vocab = set()
for text in texts:
  for word in text.lower().split():
    vocab.add(word)
vocab = list(vocab)
word_to_idx = {word: i + 1 for i, word in enumerate(vocab)}  # Start indices from 1

for text in texts:
  print(f"Original text: {text}")
  indices = text_to_indices(text, word_to_idx)
  print(f"Indices: {indices}")
  print("---")

Original text: I took the dog for a walk this morning
Indices: [17, 8, 13, 19, 3, 7, 1, 4, 9]
---
Original text: The quick brown fox jumps over the lazy dog
Indices: [13, 10, 14, 5, 20, 12, 13, 2, 19]
---
Original text: Hello how are you doing today
Indices: [15, 16, 6, 11, 18, 21]
---


## Padding Example

In [3]:
# Original sequences of different lengths
sequences = [[1, 2, 3, 4], [5, 6], [7, 8, 9]]

# Find maximum length
max_length = max(len(seq) for seq in sequences)  # 4

# Pad sequences to make them all the same length
padded_sequences = [seq + [0] * (max_length - len(seq)) for seq in sequences]
# Result: [[1, 2, 3, 4], [5, 6, 0, 0], [7, 8, 9, 0]]


## Word Embeddings Example

In [4]:
# Example vocabulary (words)
vocab = ["the", "quick", "brown", "fox", "jumps", "over", "lazy", "dog", "hello", "how", "are", "you", "doing", "today", "i", "took", "for", "a", "walk", "this", "morning"]

# Example word-to-index mapping
word_to_idx = {word: idx for idx, word in enumerate(vocab)}

# Example embedding layer
embedding_dim = 5  # You can adjust this dimension
embedding_layer = nn.Embedding(len(vocab), embedding_dim)

# Example: Get the embedding for the word "the"
word_idx = word_to_idx["the"]
word_idx_tensor = torch.tensor([word_idx])
word_embedding = embedding_layer(word_idx_tensor)

print(f"Embedding for 'the': {word_embedding}")

# Example: Get the embeddings for a sequence of words
sequence = ["the", "quick", "brown", "fox"]
sequence_indices = [word_to_idx[word] for word in sequence]
sequence_indices_tensor = torch.tensor(sequence_indices)
sequence_embeddings = embedding_layer(sequence_indices_tensor)

print(f"Embeddings for the sequence: {sequence_embeddings}")

Embedding for 'the': tensor([[-0.7140,  0.8337, -0.9585,  0.4536,  1.2461]],
       grad_fn=<EmbeddingBackward0>)
Embeddings for the sequence: tensor([[-0.7140,  0.8337, -0.9585,  0.4536,  1.2461],
        [-2.3065, -1.2869,  0.2137, -2.1268, -0.1341],
        [-1.0408, -0.7647, -0.0553,  1.2049, -0.9825],
        [ 0.3040, -0.7172,  1.0554, -1.4534,  0.4652]],
       grad_fn=<EmbeddingBackward0>)
